# 사용자의 평가경향을 고려한 CF

- 사용자의 평가 경향을 고려해서 예측치를 조정한다면 정확도가 개선될 것이라고 예상된다. 

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# 데이터 읽어 오기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('./data/u.user', sep='|', names=u_cols, encoding='latin-1')
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('./data/u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('./data/u.data', sep='\t', names=r_cols, encoding='latin-1')

# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)
# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]

# train, test 데이터 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

# 정확도(RMSE)를 계산하는 함수 
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수 
def score(model, neighbor_size=0):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs]) # change
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

In [3]:
# train set 사용자들의 Cosine similarities 계산
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

### train 데이터의 user의 rating 평균과 영화의 평점편차 계산 

In [4]:
rating_mean = rating_matrix.mean(axis=1) # full matrix에서 각 사용자의 평점 평균
rating_bias = (rating_matrix.T - rating_mean).T # 영화 평점과 각 사용자의 평균과의 차이를 구한다 

###  CF 알고리즘 (사용자의 평가 경향 고려)

In [6]:
def CF_knn_bias(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_bias: # change
        sim_scores = user_similarity[user_id].copy() 
        movie_ratings = rating_bias[movie_id].copy() # change
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index 
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx) 
        if neighbor_size == 0:          
            mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum() 
        else:                       
            if len(sim_scores) > 1: 
                neighbor_size = min(neighbor_size, len(sim_scores)) 
                sim_scores = np.array(sim_scores) 
                movie_ratings = np.array(movie_ratings)
                user_idx = np.argsort(sim_scores) 
                sim_scores = sim_scores[user_idx][-neighbor_size:] 
                movie_ratings = movie_ratings[user_idx][-neighbor_size:] 
                # 편차로 예측치 계산
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum() # change
                # 예측값에 현 사용자의 평균 더하기
                prediction = prediction + rating_mean[user_id]  # change
            else:
                prediction = rating_mean[user_id]  # change
    else:
        prediction = rating_mean[user_id] # change
    return prediction  # change

# 정확도 계산
score(CF_knn_bias, neighbor_size=30)

0.9434787597186768

### 주어진 사용자에 대해 추천을 받기

In [7]:
rating_matrix = ratings.pivot_table(values='rating', index='user_id', columns='movie_id')

matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

def recom_movie(user_id, n_items, neighbor_size=30):
    user_movie = rating_bias.loc[user_id].copy() # change
    for movie in rating_bias: # change
        # 현 사용자가 이미 평가한 영화는 제외 (평점을 0으로)        
        if pd.notnull(user_movie.loc[movie]):
            user_movie.loc[movie] = 0
        # 현 사용자가 평가하지 않은 영화의 예상 평점 계산
        else:
            user_movie.loc[movie] = CF_knn_bias(user_id, movie, neighbor_size)
    # 영화를 예상 평점에 따라 정렬해서 제목을 뽑아서 돌려 줌
    movie_sort = user_movie.sort_values(ascending=False)[:n_items] # 내림차순으로 상위의 영화를 추천 
    recom_movies = movies.loc[movie_sort.index]
    recommendations = recom_movies['title']
    return recommendations

# 2번 사용자에 대해서 5개의 추천 영화를 CF 알고리즘 (이웃 크기 30)으로 설정 
recom_movie(user_id=2, n_items=5, neighbor_size=30)

movie_id
1628                    Nico Icon (1995)
1293    Ayn Rand: A Sense of Life (1997)
1449              Golden Earrings (1947)
1639              Eighth Day, The (1996)
1398        Stranger in the House (1997)
Name: title, dtype: object

- 편향을 추가했을 때 RMSE는 약 0.9434 로 1.01833의 보다 크게 개선되었으며, 주어진 사용자에 대한 추천도 다른 값이 나오는 것을 볼 수 있다 